In [15]:
# 채권 분석자료 Crawling

import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd
from IPython.display import clear_output
    
start_page = 1
current_page = start_page
end_page = 126

result_df = pd.DataFrame(columns=['date', 'text'])

while True:
    URL = 'https://finance.naver.com/research/debenture_list.nhn?&page={}'.format(current_page)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text, 'html.parser')
    results = sp.select('table.type_1 td.file')

    for i, result in enumerate(results):
        pdf_date = sp.select('table.type_1 td.date')[i*2].text
        pdf_temp = requests.get(result.select_one('a').attrs['href'])

        with open('bond.pdf', 'wb') as f:
            f.write(pdf_temp.content)
        
        parsed = parser.from_file('bond.pdf')
        temp_text = str(parsed['content']).strip()
        temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
        result_df = pd.concat([result_df, temp_df]) 
        
    clear_output(wait=True)
    print('크롤링 {}% 진행 중...'.format(int((current_page/end_page)*100)))
        
    if current_page == end_page:
        clear_output(wait=True)
        print('데이터 크롤링 완료 ! bond_datas.csv에서 확인 가능')
        result_df.to_csv('bond_datas.csv')
        break
    current_page += 1

데이터 크롤링 완료 ! bond_datas.csv에서 확인 가능


In [19]:
# result_df['text'][0]

In [2]:
bond_df = pd.read_csv('bond_datas.csv')